In [3]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns

# to display all columns
pd.set_option('display.max_columns', None)

# To display 50 rows 
pd.set_option('display.max_rows', 50)

# Importing the movie dataset

movie = pd.read_csv('../data/Original_CMU/movie.metadata.tsv', sep= '\t', header=None)

movie.columns = ['WikiID', 'FreebaseID', 'Name', 'ReleaseDate',
                     'Revenue', 'Runtime', 'Languages_tuple', 'Countries_tuples', 'Genres_tuples']

movie.head()

,WikiID,FreebaseID,Name,ReleaseDate,Revenue,Runtime,Languages_tuple,Countries_tuples,Genres_tuples
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"


In [4]:
print(movie.shape)
print(movie.dtypes)

(81741, 9)
WikiID                int64
FreebaseID           object
Name                 object
ReleaseDate          object
Revenue             float64
Runtime             float64
Languages_tuple      object
Countries_tuples     object
Genres_tuples        object
dtype: object


In [5]:
#a lot of missing values 

print((pd.isna(movie.iloc[:,:6]).sum())/len(movie))
print((movie.iloc[:, 6:] == '{}').sum()/len(movie))

WikiID         0.000000
FreebaseID     0.000000
Name           0.000000
ReleaseDate    0.084437
Revenue        0.897224
Runtime        0.250180
dtype: float64
Languages_tuple     0.169633
Countries_tuples    0.099754
Genres_tuples       0.028064
dtype: float64


In [6]:
#The tuple in the dataset are dictionary in string format.
# Need a function to convert them to dictionary
import ast

def conv_to_dict(val): 
    try:
        return ast.literal_eval(val) if pd.notna(val) else {}
    except (ValueError, SyntaxError):
        return {}
    
movie['Languages'] = movie['Languages_tuple'].apply(conv_to_dict)
movie['Countries'] = movie['Countries_tuples'].apply(conv_to_dict)
movie['Genres'] = movie['Genres_tuples'].apply(conv_to_dict)    

In [7]:
# Now we can work directly with the dictionaries inside the columns
#Ex listing the languages for each movie
movie['Languages'].apply(lambda x: list(x.values())).head(25)

0                  [English Language]
1                  [English Language]
2                [Norwegian Language]
3                  [English Language]
4                   [German Language]
5     [Silent film, English Language]
6                  [English Language]
7                  [English Language]
8                  [Spanish Language]
9                  [English Language]
10                 [English Language]
11                [Japanese Language]
12                 [English Language]
13                 [English Language]
14                 [Turkish Language]
15                 [English Language]
16                                 []
17                 [English Language]
18                  [German Language]
19                 [English Language]
20                 [English Language]
21                 [English Language]
22                                 []
23                  [German Language]
24                 [Russian Language]
Name: Languages, dtype: object

In [8]:
#importing the movies plot dataset

plots = pd.read_csv('../data/Original_CMU/plot_summaries.txt', sep='\t', header=None)
plots.columns = ['WikiID', 'Plot']

plots.head()

,WikiID,Plot
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha..."
1,31186339,The nation of Panem consists of a wealthy Capi...
2,20663735,Poovalli Induchoodan is sentenced for six yea...
3,2231378,"The Lemon Drop Kid , a New York City swindler,..."
4,595909,Seventh-day Adventist Church pastor Michael Ch...


In [9]:
#importing the character dataset

characters = pd.read_csv('../data/Original_CMU/character.metadata.tsv', sep= '\t', header=None)
characters.columns = ['WikiMovieID', 'FreebaseMovieID', 'MovieReleaseDate','CharacterName',
                      'ActorBirthDate', 'ActorGender', 'ActorHeight','ActorEthnicity', 
                      'ActorName', 'ActorAge', 'CharacterActorMap', 'CharacterID', 'ActorID']

characters.head()

,WikiMovieID,FreebaseMovieID,MovieReleaseDate,CharacterName,ActorBirthDate,ActorGender,ActorHeight,ActorEthnicity,ActorName,ActorAge,CharacterActorMap,CharacterID,ActorID
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4
2,975900,/m/03vyhn,2001-08-24,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l
3,975900,/m/03vyhn,2001-08-24,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc
4,975900,/m/03vyhn,2001-08-24,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg


In [10]:
pd.isna(characters).sum()/len(characters)

WikiMovieID          0.000000
FreebaseMovieID      0.000000
MovieReleaseDate     0.022178
CharacterName        0.572205
ActorBirthDate       0.235528
ActorGender          0.101203
ActorHeight          0.656457
ActorEthnicity       0.764665
ActorName            0.002725
ActorAge             0.350841
CharacterActorMap    0.000000
CharacterID          0.572183
ActorID              0.001808
dtype: float64

In [11]:
#import the name clusters

name_clusters = pd.read_csv('../data/Original_CMU/name.clusters.txt', sep='\t', header=None)

name_clusters.columns = ['Instaces', 'CharacterActorMap']

print(name_clusters.shape)
name_clusters.head()

(2666, 2)


,Instaces,CharacterActorMap
0,Stuart Little,/m/0k3w9c
1,Stuart Little,/m/0k3wcx
2,Stuart Little,/m/0k3wbn
3,John Doe,/m/0jyg35
4,John Doe,/m/0k2_zn


In [12]:
#import the tvtropes clusters

tvtropes_clusters = pd.read_csv('../data/Original_CMU/tvtropes.clusters.txt', sep='\t', header=None)
tvtropes_clusters.columns = ['Instances', 'Dictionary_str']

tvtropes_clusters.head()

,Instances,Dictionary_str
0,absent_minded_professor,"{""char"": ""Professor Philip Brainard"", ""movie"":..."
1,absent_minded_professor,"{""char"": ""Professor Keenbean"", ""movie"": ""Richi..."
2,absent_minded_professor,"{""char"": ""Dr. Reinhardt Lane"", ""movie"": ""The S..."
3,absent_minded_professor,"{""char"": ""Dr. Harold Medford"", ""movie"": ""Them!..."
4,absent_minded_professor,"{""char"": ""Daniel Jackson"", ""movie"": ""Stargate""..."


In [13]:
tvtropes_clusters['Dictionary'] = tvtropes_clusters['Dictionary_str'].apply(conv_to_dict)

#create column for each key in the dictionary

c = tvtropes_clusters['Dictionary'].apply(pd.Series)

tvtropes_new = pd.concat([tvtropes_clusters['Instances'], c], axis=1)
tvtropes_new.rename(columns= {'id': 'CharacterActorMap'}, inplace=True)
tvtropes_new.head()

,Instances,char,movie,CharacterActorMap,actor
0,absent_minded_professor,Professor Philip Brainard,Flubber,/m/0jy9q0,Robin Williams
1,absent_minded_professor,Professor Keenbean,Richie Rich,/m/02vchl3,Michael McShane
2,absent_minded_professor,Dr. Reinhardt Lane,The Shadow,/m/0k6fkc,Ian McKellen
3,absent_minded_professor,Dr. Harold Medford,Them!,/m/0k6_br,Edmund Gwenn
4,absent_minded_professor,Daniel Jackson,Stargate,/m/0k3rhh,James Spader


In [14]:
#Merging plot with movie
#plots dataset have only 40k compare to the 80k in the movie dataset

movie_plot = movie.merge(plots, on = 'WikiID', how = 'left')
movie_plot.head()

,WikiID,FreebaseID,Name,ReleaseDate,Revenue,Runtime,Languages_tuple,Countries_tuples,Genres_tuples,Languages,Countries,Genres,Plot
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",{'/m/02h40lc': 'English Language'},{'/m/09c7w0': 'United States of America'},"{'/m/01jfsb': 'Thriller', '/m/06n90': 'Science...","Set in the second half of the 22nd century, th..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp...",{'/m/02h40lc': 'English Language'},{'/m/09c7w0': 'United States of America'},"{'/m/02n4kr': 'Mystery', '/m/03bxz7': 'Biograp...",NaN
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D...",{'/m/05f_3': 'Norwegian Language'},{'/m/05b4w': 'Norway'},"{'/m/0lsxr': 'Crime Fiction', '/m/07s9rl0': 'D...",NaN
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic...",{'/m/02h40lc': 'English Language'},{'/m/07ssc': 'United Kingdom'},"{'/m/01jfsb': 'Thriller', '/m/0glj9q': 'Erotic...",A series of murders of rich young women throug...
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}",{'/m/04306rv': 'German Language'},{'/m/0345h': 'Germany'},{'/m/07s9rl0': 'Drama'},"Eva, an upper class housewife, becomes frustra..."


In [15]:
#for each caracter associate his tvtropes type 
#there are only 501 character/actor pair that have an associated tvtropse type, some character/actor pair have multiple tvtropes type

#es
print(tvtropes_new['CharacterActorMap'].value_counts())
print(tvtropes_new[tvtropes_new['CharacterActorMap'] == '/m/0k1xxm'])

#We need to group the instances by the character/actor pair

CharacterActorMap
/m/0k1xxm     5
/m/02hw79s    4
/m/0j_fvv     3
/m/0j1__hv    3
/m/0k65z5     3
             ..
/m/0k79b3     1
/m/0k60j5     1
/m/0jx21w     1
/m/0jy52w     1
/m/0k39jj     1
Name: count, Length: 447, dtype: int64
          Instances                  char  \
73     byronic_hero  Captain Jack Sparrow   
95          charmer  Captain Jack Sparrow   
132          coward  Captain Jack Sparrow   
339  loveable_rogue  Captain Jack Sparrow   
476       trickster  Captain Jack Sparrow   

                                          movie CharacterActorMap        actor  
73   Pirates of the Caribbean: Dead Man's Chest         /m/0k1xxm  Johnny Depp  
95   Pirates of the Caribbean: Dead Man's Chest         /m/0k1xxm  Johnny Depp  
132  Pirates of the Caribbean: Dead Man's Chest         /m/0k1xxm  Johnny Depp  
339  Pirates of the Caribbean: Dead Man's Chest         /m/0k1xxm  Johnny Depp  
476  Pirates of the Caribbean: Dead Man's Chest         /m/0k1xxm  Johnny Depp  


In [16]:
grouped_instances = tvtropes_new.groupby('CharacterActorMap').agg({'Instances' : lambda x: x.tolist()}).reset_index()

print(grouped_instances['CharacterActorMap'].is_unique)
grouped_instances[grouped_instances['CharacterActorMap'] == '/m/0k1xxm']

True


,CharacterActorMap,Instances
316,/m/0k1xxm,"[byronic_hero, charmer, coward, loveable_rogue..."


In [20]:
character_instances = characters.merge(grouped_instances, on = 'CharacterActorMap', how = 'left')
print(character_instances.shape)

(450669, 14)


In [18]:
grouped_char = character_instances.groupby('WikiMovieID').agg({
    'CharacterName': lambda x: {i + 1: val for i, val in enumerate(x.tolist())},
    'CharacterID': lambda x: {i + 1: val for i, val in enumerate(x.tolist())},
    'Instances': lambda x: {i + 1: val for i, val in enumerate(x.tolist())},
    'CharacterActorMap': lambda x: {i + 1: val for i, val in enumerate(x.tolist())},
    'ActorID': lambda x: {i + 1: val for i, val in enumerate(x.tolist())},
    'ActorName': lambda x: {i + 1: val for i, val in enumerate(x.tolist())},
    'ActorAge': lambda x: {i + 1: val for i, val in enumerate(x.tolist())},
    'ActorGender': lambda x: {i + 1: val for i, val in enumerate(x.tolist())},
    'ActorBirthDate' : lambda x: {i + 1: val for i, val in enumerate(x.tolist())},
    'ActorHeight' : lambda x: {i + 1: val for i, val in enumerate(x.tolist())},
}).reset_index()


grouped_char.head()

,WikiMovieID,CharacterName,CharacterID,Instances,CharacterActorMap,ActorID,ActorName,ActorAge,ActorGender,ActorBirthDate,ActorHeight
0,330,"{1: nan, 2: nan, 3: nan, 4: nan}","{1: nan, 2: nan, 3: nan, 4: nan}","{1: nan, 2: nan, 3: nan, 4: nan}","{1: '/m/02vbt4w', 2: '/m/02vb4j6', 3: '/m/02vc...","{1: '/m/0gh6sw', 2: '/m/0267qhz', 3: '/m/02634...","{1: 'Rosa Maria Sardà', 2: 'Mercè Pons', 3: 'A...","{1: 54.0, 2: 29.0, 3: 51.0, 4: 60.0}","{1: 'F', 2: 'F', 3: 'F', 4: 'F'}","{1: '1941-07-30', 2: '1966', 3: '1944-08-31', ...","{1: nan, 2: nan, 3: nan, 4: nan}"
1,3217,"{1: 'S-Mart Clerk', 2: 'Fake shemp', 3: 'Ash W...","{1: '/m/0hgcnkq', 2: '/m/089hc6', 3: '/m/06_kr...","{1: nan, 2: nan, 3: nan, 4: nan, 5: nan, 6: na...","{1: '/m/0hgcnkm', 2: '/m/04hzm56', 3: '/m/0jtq...","{1: '/m/07qn0', 2: '/m/0gmjf8', 3: '/m/01h8f',...","{1: 'Ted Raimi', 2: 'Ivan Raimi', 3: 'Bruce Ca...","{1: 26.0, 2: 36.0, 3: 34.0, 4: 34.0, 5: 26.0, ...","{1: 'M', 2: 'M', 3: 'M', 4: 'M', 5: 'M', 6: 'F...","{1: '1965-12-14', 2: '1956', 3: '1958-06-22', ...","{1: 1.87, 2: nan, 3: 1.85, 4: 1.85, 5: 1.87, 6..."
2,3333,"{1: nan, 2: nan, 3: nan, 4: nan, 5: nan, 6: na...","{1: nan, 2: nan, 3: nan, 4: nan, 5: nan, 6: na...","{1: nan, 2: nan, 3: nan, 4: nan, 5: nan, 6: na...","{1: '/m/0k4hhp', 2: '/m/0k4hhv', 3: '/m/0k4hh_...","{1: '/m/01c2sl', 2: '/m/0bkrvh', 3: '/m/0735gw...","{1: 'Lillian Gish', 2: 'Henry B. Walthall', 3:...","{1: nan, 2: nan, 3: nan, 4: nan, 5: nan, 6: 11...","{1: 'F', 2: 'M', 3: 'F', 4: 'M', 5: 'M', 6: 'F...","{1: '1893-10-14', 2: '1878-03-16', 3: '1894-11...","{1: 1.66, 2: nan, 3: 1.6, 4: nan, 5: nan, 6: n..."
3,3746,"{1: 'J.F. Sebastian', 2: 'Rick Deckard', 3: 'R...","{1: '/m/02nwbsh', 2: '/m/08zbgl', 3: '/m/02nb0...","{1: nan, 2: ['bounty_hunter', 'hardboiled_dete...","{1: '/m/021lq_w', 2: '/m/0jvhbm', 3: '/m/0jvhb...","{1: '/m/05wqr1', 2: '/m/0c0k1', 3: '/m/09byk',...","{1: 'William Sanderson', 2: 'Harrison Ford', 3...","{1: 34.0, 2: 39.0, 3: 38.0, 4: 22.0, 5: 21.0, ...","{1: 'M', 2: 'M', 3: 'M', 4: 'F', 5: 'F', 6: 'F...","{1: '1948-01-10', 2: '1942-07-13', 3: '1944-01...","{1: 1.727, 2: 1.85, 3: 1.85, 4: 1.753, 5: 1.78..."
4,3837,"{1: 'Gabby Johnson', 2: 'Taggart', 3: nan, 4: ...","{1: '/m/0h5rd8f', 2: '/m/04w0vk1', 3: nan, 4: ...","{1: nan, 2: nan, 3: nan, 4: nan, 5: ['dumb_mus...","{1: '/m/0cg0_v5', 2: '/m/05nn179', 3: '/m/05n7...","{1: '/m/06l6sp', 2: '/m/0hn04', 3: '/m/02x5ztx...","{1: 'Jack Starrett', 2: 'Slim Pickens', 3: 'Da...","{1: 37.0, 2: 54.0, 3: 64.0, 4: 57.0, 5: 38.0, ...","{1: 'M', 2: 'M', 3: 'M', 4: 'M', 5: 'M', 6: 'M...","{1: '1936-11-02', 2: '1919-06-29', 3: '1910-02...","{1: nan, 2: 1.91, 3: nan, 4: 1.7, 5: 1.88, 6: ..."


In [21]:
grouped_char['WikiMovieID'].is_unique

True

In [22]:
#Now we can merge the character_instances with the movie_plot dataset

movie_plot_char = movie_plot.merge(grouped_char, left_on = 'WikiID', right_on = 'WikiMovieID', how = 'left')
print(movie_plot_char.shape)

(81741, 24)


In [23]:
movie_plot_char.head()

,WikiID,FreebaseID,Name,ReleaseDate,Revenue,Runtime,Languages_tuple,Countries_tuples,Genres_tuples,Languages,Countries,Genres,Plot,WikiMovieID,CharacterName,CharacterID,Instances,CharacterActorMap,ActorID,ActorName,ActorAge,ActorGender,ActorBirthDate,ActorHeight
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",{'/m/02h40lc': 'English Language'},{'/m/09c7w0': 'United States of America'},"{'/m/01jfsb': 'Thriller', '/m/06n90': 'Science...","Set in the second half of the 22nd century, th...",975900.0,"{1: 'Akooshay', 2: 'Lieutenant Melanie Ballard...","{1: '/m/0bgcj3x', 2: '/m/0bgchn4', 3: '/m/0bgc...","{1: nan, 2: nan, 3: nan, 4: nan, 5: nan, 6: na...","{1: '/m/0bgchxw', 2: '/m/0jys3m', 3: '/m/0jys3...","{1: '/m/03wcfv7', 2: '/m/0346l4', 3: '/m/01vw2...","{1: 'Wanda De Jesus', 2: 'Natasha Henstridge',...","{1: 42.0, 2: 27.0, 3: 32.0, 4: 33.0, 5: 23.0, ...","{1: 'F', 2: 'F', 3: 'M', 4: 'M', 5: 'F', 6: 'F...","{1: '1958-08-26', 2: '1974-08-15', 3: '1969-06...","{1: 1.62, 2: 1.78, 3: 1.727, 4: 1.75, 5: 1.65,..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp...",{'/m/02h40lc': 'English Language'},{'/m/09c7w0': 'United States of America'},"{'/m/02n4kr': 'Mystery', '/m/03bxz7': 'Biograp...",NaN,3196793.0,"{1: 'Police Officer', 2: 'Reporter', 3: nan, 4...","{1: '/m/0lr3z80', 2: '/m/0lr3z8g', 3: nan, 4: ...","{1: nan, 2: nan, 3: nan, 4: nan, 5: nan, 6: na...","{1: '/m/0lr37fl', 2: '/m/0lr37d8', 3: '/m/0lr3...","{1: '/m/0h8pnlw', 2: '/m/034dbm', 3: '/m/0lr37...","{1: 'Allen Cutler', 2: 'Alice Barrett', 3: 'Ro...","{1: nan, 2: 43.0, 3: nan, 4: nan, 5: 50.0, 6: ...","{1: 'M', 2: 'F', 3: 'M', 4: 'F', 5: 'M', 6: 'M...","{1: nan, 2: '1956-12-19', 3: nan, 4: nan, 5: '...","{1: nan, 2: nan, 3: nan, 4: nan, 5: nan, 6: 1...."
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D...",{'/m/05f_3': 'Norwegian Language'},{'/m/05b4w': 'Norway'},"{'/m/0lsxr': 'Crime Fiction', '/m/07s9rl0': 'D...",NaN,28463795.0,"{1: nan, 2: nan, 3: nan, 4: nan}","{1: nan, 2: nan, 3: nan, 4: nan}","{1: nan, 2: nan, 3: nan, 4: nan}","{1: '/m/0gct1bn', 2: '/m/0gct1bv', 3: '/m/0gct...","{1: '/m/053j7xf', 2: '/m/0gct1by', 3: '/m/0bwh...","{1: 'Frank Krog', 2: 'Kristin Kajander', 3: 'V...","{1: 33.0, 2: 28.0, 3: 40.0, 4: 30.0}","{1: 'M', 2: 'F', 3: 'M', 4: 'F'}","{1: '1954-10-05', 2: '1959-02-28', 3: '1947', ...","{1: nan, 2: nan, 3: nan, 4: nan}"
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic...",{'/m/02h40lc': 'English Language'},{'/m/07ssc': 'United Kingdom'},"{'/m/01jfsb': 'Thriller', '/m/0glj9q': 'Erotic...",A series of murders of rich young women throug...,9363483.0,"{1: nan, 2: nan}","{1: nan, 2: nan}","{1: nan, 2: nan}","{1: '/m/02t9_0c', 2: '/m/02vct27'}","{1: '/m/036496', 2: '/m/07g60h'}","{1: 'David Keith', 2: 'Cathy Moriarty'}","{1: 32.0, 2: 26.0}","{1: 'M', 2: 'F'}","{1: '1954-05-08', 2: '1960-11-29'}","{1: 1.83, 2: 1.75}"
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}",{'/m/04306rv': 'German Language'},{'/m/0345h': 'Germany'},{'/m/07s9rl0': 'Drama'},"Eva, an upper class housewife, becomes frustra...",261236.0,"{1: nan, 2: nan, 3: nan}","{1: nan, 2: nan, 3: nan}","{1: nan, 2: nan, 3: nan}","{1: '/m/02vb3cv', 2: '/m/02tbd9f', 3: '/m/02vd...","{1: '/m/09d6hv', 2: '/m/06prxs', 3: '/m/09k3x_'}","{1: 'Gudrun Landgrebe', 2: 'Mathieu Carrière',...","{1: 32.0, 2: 32.0, 3: 35

In [25]:
# load ratings dataset

ratings = pd.read_csv('../data/Movie_Ratings/ratings.csv')
ratings.shape

(32000204, 4)

In [26]:
m_ratings = pd.read_csv('../data/Movie_Ratings/movies.csv')
m_ratings.shape

(87585, 3)

In [27]:
grouped_ratings = ratings.groupby('movieId')['rating'].agg(['mean', 'std', 'count']).reset_index()

grouped_ratings.shape

(84432, 4)

In [28]:
name_ratings = m_ratings.iloc[:,:2].merge(grouped_ratings, on = 'movieId', how = 'left')
name_ratings.head()

,movieId,title,mean,std,count
0,1,Toy Story (1995),3.897438,0.921450,68997.0
1,2,Jumanji (1995),3.275758,0.955544,28904.0
2,3,Grumpier Old Men (1995),3.139447,1.012273,13134.0
3,4,Waiting to Exhale (1995),2.845331,1.105871,2806.0
4,5,Father of the Bride Part II (1995),3.059602,0.999079,13154.0


In [24]:
# [df_movies['merge_title'] = df_movies['title'].str.extract(r'^(.*?)(?=\s\()')[0].str.lower()
# df_movies['merge_year'] = df_movies['title'].str.extract(r'\((\d{4})\)')[0].astype('Int64')
# [14:09, 11/11/2024] +49 1522 4688950: df_movie_metadata['merge_title'] = df_movie_metadata['Movie name'].str.lower().str.strip()
# df_movie_metadata['merge_year'] = df_movie_metadata['Movie release date'].apply(lambda x: x.split('-')[0] if pd.notna(x) else None).astype('Int64')
# [14:10, 11/11/2024] +49 1522 4688950: df_merged = pd.merge(df_movie_metadata, df_movies, how='inner', on=['merge_title', 'merge_year'])

In [29]:
name_ratings['merge_title'] = name_ratings['title'].str.extract(r'^(.*?)(?=\s\()')[0].str.lower()
name_ratings['merge_year'] = name_ratings['title'].str.extract(r'\((\d{4})\)')[0].astype('Int64')

movie_plot_char['merge_title'] = movie_plot_char['Name'].str.lower().str.strip()
movie_plot_char['merge_year'] = movie_plot_char['ReleaseDate'].apply(lambda x: x.split('-')[0] if pd.notna(x) else None).astype('Int64')

df_final = movie_plot_char.merge(name_ratings, on = ['merge_title', 'merge_year'], how = 'left')

In [30]:
df_final.head()

,WikiID,FreebaseID,Name,ReleaseDate,Revenue,Runtime,Languages_tuple,Countries_tuples,Genres_tuples,Languages,Countries,Genres,Plot,WikiMovieID,CharacterName,CharacterID,Instances,CharacterActorMap,ActorID,ActorName,ActorAge,ActorGender,ActorBirthDate,ActorHeight,merge_title,merge_year,movieId,title,mean,std,count
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",{'/m/02h40lc': 'English Language'},{'/m/09c7w0': 'United States of America'},"{'/m/01jfsb': 'Thriller', '/m/06n90': 'Science...","Set in the second half of the 22nd century, th...",975900.0,"{1: 'Akooshay', 2: 'Lieutenant Melanie Ballard...","{1: '/m/0bgcj3x', 2: '/m/0bgchn4', 3: '/m/0bgc...","{1: nan, 2: nan, 3: nan, 4: nan, 5: nan, 6: na...","{1: '/m/0bgchxw', 2: '/m/0jys3m', 3: '/m/0jys3...","{1: '/m/03wcfv7', 2: '/m/0346l4', 3: '/m/01vw2...","{1: 'Wanda De Jesus', 2: 'Natasha Henstridge',...","{1: 42.0, 2: 27.0, 3: 32.0, 4: 33.0, 5: 23.0, ...","{1: 'F', 2: 'F', 3: 'M', 4: 'M', 5: 'F', 6: 'F...","{1: '1958-08-26', 2: '1974-08-15', 3: '1969-06...","{1: 1.62, 2: 1.78, 3: 1.727, 4: 1.75, 5: 1.65,...",ghosts of mars,2001,4735.0,Ghosts of Mars (2001),2.35196,1.064942,1709.0
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp...",{'/m/02h40lc': 'English Language'},{'/m/09c7w0': 'United States of America'},"{'/m/02n4kr': 'Mystery', '/m/03bxz7': 'Biograp...",NaN,3196793.0,"{1: 'Police Officer', 2: 'Reporter', 3: nan, 4...","{1: '/m/0lr3z80', 2: '/m/0lr3z8g', 3: nan, 4: ...","{1: nan, 2: nan, 3: nan, 4: nan, 5: nan, 6: na...","{1: '/m/0lr37fl', 2: '/m/0lr37d8', 3: '/m/0lr3...","{1: '/m/0h8pnlw', 2: '/m/034dbm', 3: '/m/0lr37...","{1: 'Allen Cutler', 2: 'Alice Barrett', 3: 'Ro...","{1: nan, 2: 43.0, 3: nan, 4: nan, 5: 50.0, 6: ...","{1: 'M', 2: 'F', 3: 'M', 4: 'F', 5: 'M', 6: 'M...","{1: nan, 2: '1956-12-19', 3: nan, 4: nan, 5: '...","{1: nan, 2: nan, 3: nan, 4: nan, 5: nan, 6: 1....",getting away with murder: the jonbenét ramsey ...,2000,NaN,NaN,NaN,NaN,NaN
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D...",{'/m/05f_3': 'Norwegian Language'},{'/m/05b4w': 'Norway'},"{'/m/0lsxr': 'Crime Fiction', '/m/07s9rl0': 'D...",NaN,28463795.0,"{1: nan, 2: nan, 3: nan, 4: nan}","{1: nan, 2: nan, 3: nan, 4: nan}","{1: nan, 2: nan, 3: nan, 4: nan}","{1: '/m/0gct1bn', 2: '/m/0gct1bv', 3: '/m/0gct...","{1: '/m/053j7xf', 2: '/m/0gct1by', 3: '/m/0bwh...","{1: 'Frank Krog', 2: 'Kristin Kajander', 3: 'V...","{1: 33.0, 2: 28.0, 3: 40.0, 4: 30.0}","{1: 'M', 2: 'F', 3: 'M', 4: 'F'}","{1: '1954-10-05', 2: '1959-02-28', 3: '1947', ...","{1: nan, 2: nan, 3: nan, 4: nan}",brun bitter,1988,NaN,NaN,NaN,NaN,NaN
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic...",{'/m/02h40lc': 'English Language'},{'/m/07ssc': 'United Kingdom'},"{'/m/01jfsb': 'Thriller', '/m/0glj9q': 'Erotic...",A series of murders of rich young women throug...,9363483.0,"{1: nan, 2: nan}","{1: nan, 2: nan}","{1: nan, 2: nan}","{1: '/m/02t9_0c', 2: '/m/02vct27'}","{1: '/m/036496', 2: '/m/07g60h'}","{1: 'David Keith', 2: 'Cathy Moriarty'}","{1: 32.0, 2: 26.0}","{1: 'M', 2: 'F'}","{1: '1954-05-08', 2: '1960-11-29'}","{1: 1.83, 2: 1.75}",white of the eye,1987,90015.0,White of the Eye (1987),3.00000,1.038328,33.0
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}",{'/m/04306rv': 'German Language'},{'/m/0345h': 'Germany'},{'/m/07s9rl0': 'Drama'},"Eva, an 

In [31]:
print(df_final.isna().sum()/len(df_final))

WikiID               0.000000
FreebaseID           0.000000
Name                 0.000000
ReleaseDate          0.084367
Revenue              0.897041
Runtime              0.250021
Languages_tuple      0.000000
Countries_tuples     0.000000
Genres_tuples        0.000000
Languages            0.000000
Countries            0.000000
Genres               0.000000
Plot                 0.483516
WikiMovieID          0.212862
CharacterName        0.212862
CharacterID          0.212862
Instances            0.212862
CharacterActorMap    0.212862
ActorID              0.212862
ActorName            0.212862
ActorAge             0.212862
ActorGender          0.212862
ActorBirthDate       0.212862
ActorHeight          0.212862
merge_title          0.000000
merge_year           0.084367
movieId              0.709800
title                0.709800
mean                 0.720275
std                  0.755369
count                0.720275
dtype: float64


In [34]:
df_final.to_csv('../data/processed/df_final.csv', index=False)